In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
path = Path('../datasources/JoinedViscosityNew.csv')

df = pd.read_csv(path)

# # TEMP
# df = pd.read_csv(base_path / 'errors.csv')
# df = df[df['errors'].isnull()]

# df.to_csv('data.csv')

In [ ]:
# df['IE'].hist()
# plt.show()
# df['EA'].hist()
# plt.show()

In [ ]:
# processed_df = df.copy()

# processed_df = processed_df[processed_df['IE'].between(-2, 10)]
# processed_df = processed_df[processed_df['EA'].between(-2, 10)]
# processed_df = processed_df[~processed_df['smiles'].str.contains('/')]


# processed_df['IE'].hist(bins=30)
# plt.show()
# processed_df['EA'].hist(bins=30)
# plt.show()

# print(len(processed_df.index))
# processed_df.to_csv('data.csv', index=False)

In [7]:
%load_ext autoreload
%autoreload

from grammar import old_grammar

grammar = old_grammar

df['error'] = np.nan

for id, row in df.iterrows():
    smi = row['Cation']
    try:
        grammar.parse_smiles_list([smi])
    except Exception as e:
        df.loc[id, 'error'] = e
        print(smi, ' | ', e)


for id, row in df.iterrows():
    smi = row['Anion']
    try:
        grammar.parse_smiles_list([smi])
    except Exception as e:
        df.loc[id, 'error'] = e
        print(smi, ' | ', e)

df = df.sort_values(by='error')
df.to_csv('errors.csv')

df = df[df['error'].isna()]
df.to_csv('viscosity.csv')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
C[NH+]1CN(C=C1)CCCCCCCC\C=C\CCCCCCCC  |  Grammar does not cover some of the input words: "'\\\\', '\\\\'".
nan  |  'float' object has no attribute 'replace'
[Li+]  |  Grammar does not cover some of the input words: "'L', 'i'".
[K+]  |  Grammar does not cover some of the input words: "'K'".
C[NH+]1CN(C=C1)CCCCCCCC\C=C/C\C=C/CCCCC  |  Grammar does not cover some of the input words: "'\\\\', '/', '\\\\', '/'".
C(CCCCCCC\C=C/CCCCCCCC)[N+]1=CN(C=C1)C  |  Grammar does not cover some of the input words: "'\\\\', '/'".
C(CCCCCCC\C=C/CCCCCCCC)(=O)[O-]  |  Grammar does not cover some of the input words: "'\\\\', '/'".
C/C(/C(=O)[O-])=C\C  |  Grammar does not cover some of the input words: "'/', '/', '\\\\'".
C(CCCCCCC\C=C/CCCCCCCC)(=O)[O-]  |  Grammar does not cover some of the input words: "'\\\\', '/'".
C(CCCCCCC\C=C/CCCCCCCC)(=O)[O-]  |  Grammar does not cover some of the input words: "'\\\\', '/'".
[Re](=

In [24]:
# df[df['errors'].notnull()]
errors_df = df.groupby('errors')['smiles'].nunique().reset_index()
errors_df = errors_df.sort_values('smiles', ascending=False).reset_index()
errors_df.to_csv('group_errors.csv')

In [56]:
unknown_df = errors_df[errors_df['errors'].str.startswith('Grammar does not cover some of the input words: ')]
unknown_df['errors'] = unknown_df['errors'].str.removeprefix('Grammar does not cover some of the input words: "')
unknown_df['errors'] = unknown_df['errors'].str.removesuffix('".')
unknown_df['errors'] = unknown_df['errors'].str.split(', ').map(lambda i: [char.removeprefix("'").removesuffix("'") for char in i])



unknown_array = np.concatenate(unknown_df['errors'].to_list())
unique, counts = np.unique(unknown_array, return_counts=True)

dict(zip(unique, counts))

/tmp/ipykernel_695/1057138089.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unknown_df['errors'] = unknown_df['errors'].str.removeprefix('Grammar does not cover some of the input words: "')
/tmp/ipykernel_695/1057138089.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unknown_df['errors'] = unknown_df['errors'].str.removesuffix('".')
/tmp/ipykernel_695/1057138089.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

{'.': 103,
 '/': 210,
 '5': 52,
 '6': 20,
 '7': 8,
 '8': 2,
 '@': 339,
 'L': 4,
 'Z': 14,
 '\\\\\\\\': 102,
 'a': 13,
 'e': 1,
 'i': 4,
 's': 40}